In [35]:
import tensorflow as tf
from PIL import Image
import numpy as np
import json

In [59]:
model = tf.saved_model.load(tags=['serve'], export_dir="./")

In [27]:
predict_fn=model.signatures["serving_default"]

In [38]:
f = open('signature.json',)
signature = json.load(f)
inputs=signature.get("inputs")
outputs=signature.get("outputs")

{'Confidences': {'dtype': 'float32',
  'shape': [None, 6],
  'name': '62da264a-5079-4029-a2a7-ac824da0be1d.3dcb847f-69e8-4aca-ac1b-872314e1d1ce/dense_2/Softmax:0'}}

In [51]:
def predict(image: Image.Image) -> dict:
    # pre-processing the image before passing to model
    image = process_image(image, inputs.get("Image").get("shape"))
    feed_dict = {}
    feed_dict[list(inputs.keys())[0]] = tf.convert_to_tensor(image)
    outputs = predict_fn(**feed_dict)
    return process_output(outputs)

In [52]:
def process_image(image, input_shape) -> np.ndarray:
    """
    center square crop and resize to fit the expected model input, and convert from [0,255] to [0,1] values.
    """
    width, height = image.size
    # ensure image type is compatible with model and convert if not
    if image.mode != "RGB":
        image = image.convert("RGB")
    # center crop image
    if width != height:
        square_size = min(width, height)
        left = (width - square_size) / 2
        top = (height - square_size) / 2
        right = (width + square_size) / 2
        bottom = (height + square_size) / 2
        # Crop the center of the image
        image = image.crop((left, top, right, bottom))
    # now the image is square, resize it to be the right shape for the model input
    input_width, input_height = input_shape[1:3]
    if image.width != input_width or image.height != input_height:
        image = image.resize((input_width, input_height))

    # make 0-1 float instead of 0-255 int (that PIL Image loads by default)
    image = np.asarray(image) / 255.0
    # pad with an extra batch dimension as expected by the model
    return np.expand_dims(image, axis=0).astype(np.float32)

In [56]:
def process_output(outputs) -> dict:
        #postprocessing
        out_keys = ["label", "confidence"]
        results = {}
        for key, tf_val in outputs.items():
            val = tf_val.numpy().tolist()[0]
            if isinstance(val, bytes):
                val = val.decode()
            results[key] = val
        confs = results["Confidences"]
        labels = signature.get("classes").get("Label")
        output = [dict(zip(out_keys, group)) for group in zip(labels, confs)]
        sorted_output = {"predictions": sorted(output, key=lambda k: k["confidence"], reverse=True)}
        return sorted_output

In [60]:
image=Image.open("./1.jpg")

In [61]:
predict(image)

{'predictions': [{'label': 'audi car images',
   'confidence': 0.9497203230857849},
  {'label': 'bmw car images', 'confidence': 0.03611782193183899},
  {'label': 'mercedes car images', 'confidence': 0.011200593784451485},
  {'label': 'maruti suzuki car images', 'confidence': 0.0023138721007853746},
  {'label': 'hyundai car images', 'confidence': 0.0006463660392910242},
  {'label': 'lamborghini car images', 'confidence': 1.133359319283045e-06}]}